# Library

In [2]:
pip install swifter

In [3]:
pip install Sastrawi

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install nltk

In [5]:
# LIBRARY UNTUK MENGOLAH DATA
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.model_selection import train_test_split

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

import time

import warnings
warnings.filterwarnings("ignore")
# LIBRARY UNTUK PREPROCESSING
import nltk #import library nltk
from nltk.tokenize import word_tokenize #import word_tokenize for tokenizing text into words 
from nltk.tokenize import sent_tokenize #import sent_tokenize for tokenizing paragraph into sentences
from nltk.stem.porter import PorterStemmer #import Porter Stemmer Algorithm 
from nltk.stem import WordNetLemmatizer #import WordNet lemmatizer 
from nltk.corpus import stopwords #import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import Indonesian Stemmer
import re,string #import regular expression

# STOPWORD DENGAN SASTRAWI
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# TOKENIZE
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mfaishaldp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mfaishaldp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data Analysis

## Read Dataset

In [14]:
dt = pd.read_csv('Data - Bismillah.csv')

In [15]:
dt['Label'].value_counts()

-1    8631
 1    7421
 0    6115
Name: Label, dtype: int64

## Data Cleaning
Proses pada tahap ini untuk membersihkan atau menghapus data dari tanda baca angka, URL, dan hastag

In [16]:
dt_prep = dt.copy()
dt_prep.isna().sum()

Date     0
Text     0
Label    0
dtype: int64

In [17]:
dt_prep

,Date,Text,Label
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0
2,2019-07-20,Bankerot nih namanya,-1
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0
...,...,...,...
22162,2019-04-02,@GdeDyaksa Terima kasih Bpk Dyaksa atas partis...,1
22163,2019-04-02,@mohammadiball Terima kasih Bpk Ibal atas part...,1
22164,2019-04-02,@JuraganHusin Terima kasih Bpk Husin atas part...,1
22165,2019-04-02,@vill4nk Terima kasih Bpk/Ibu atas partisipasi...,1


In [18]:
dup = dt_prep.duplicated(subset='Text')
dup.sum()

38

In [19]:
dt_prep.head()

,Date,Text,Label
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0
2,2019-07-20,Bankerot nih namanya,-1
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0


In [20]:
dt_prep.drop_duplicates(subset='Text', inplace = True)
dt_prep.reset_index(drop=True,inplace=True)

In [21]:
dt_prep

,Date,Text,Label
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0
2,2019-07-20,Bankerot nih namanya,-1
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0
...,...,...,...
22124,2019-04-02,@GdeDyaksa Terima kasih Bpk Dyaksa atas partis...,1
22125,2019-04-02,@mohammadiball Terima kasih Bpk Ibal atas part...,1
22126,2019-04-02,@JuraganHusin Terima kasih Bpk Husin atas part...,1
22127,2019-04-02,@vill4nk Terima kasih Bpk/Ibu atas partisipasi...,1


In [22]:
dt_prep['Label'].value_counts()

-1    8626
 1    7392
 0    6111
Name: Label, dtype: int64

### Function Prepros


In [10]:
def hapus_tanda(tweet): 
    tanda_baca = set(string.punctuation)
    tweet = ''.join(ch for ch in tweet if ch not in tanda_baca)
    return tweet
  
def hapus_katadouble(s): 
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

def cleaning(tweet): 
    regex = r'\b(\w+)(?:\W+\1\b)+' 
    tweet = re.sub(r"\n", " ",tweet) 
    tweet = re.sub(r"\r", " ",tweet)
    tweet = re.sub(r"&amp;", " ",tweet)
    tweet = re.sub(r"&gt;", " ",tweet)
    tweet = re.sub(r"\d"," ",tweet)
    tweet = re.sub(r"[_]",' ',tweet)
    
    # NEW TAMBAHAN
    # HAPUS USERNAME DAN ISINYA
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    # HAPUS HASTAG DAN ISINYA
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
    
    # hapus URL
    tweet = re.sub(r'''(?i)\b((?:https|http?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "",tweet)
    # hapus simbol
    tweet = re.sub(r"[-_<>*#$@()\"/%&;:<>{}`+=~|.!?,']", " ", tweet)
    # hapus HTML
    tweet = re.sub(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});", " ", tweet)
    # hapus ASCII
    tweet = re.sub(r"[^\x00-\x7f]", " ", tweet)
    # hapus double spasi
    tweet = re.sub("\s\s+", " ",tweet) 
    # hapus username
    tweet = re.sub('@[^\s]+','',tweet)
    # hapus #tagger 
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    # hapus tanda baca
    tweet = hapus_tanda(tweet)
    # hapus angka dan angka yang berada dalam string 
    tweet = re.sub(r'\w*\d\w*', '',tweet).strip()
    # hapus repetisi karakter 
    tweet = hapus_katadouble(tweet)
    # hapus Emoji
    emoji_pattern =  re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    tweet = emoji_pattern.sub(r'',tweet)
    # menghapus perulangan kata
    tweet = re.sub(regex, r'\1', tweet, flags=re.IGNORECASE) 
    return tweet

def tokenizing(data):
    return nltk.tokenize.word_tokenize(data)

def stemming(text, stemmer_id):
    text_split = text.split(" ")
    stemmed_list = []
    for i in text_split:
        stemmed_list.append(stemmer_id.stem(i))
    stemmed = ' '.join(map(str,stemmed_list))
    return stemmed

def stemData(data, stemmer_id):
    data = data.map(lambda com : stemming(com,stemmer_id))
    return data

# yang belum 'Case Folding' & 'Tokenizing'

### Clean Data

In [11]:
dt_prep['Text Clean'] = dt_prep['Text'].apply(cleaning)
dt_prep.head(5)

,Date,Text,Label,Text Clean
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...


In [12]:
len(dt_prep)

22129

In [13]:
dup1 = dt_prep.duplicated(subset='Text Clean')
dup1.sum()

1604

In [14]:
dt_prep.drop_duplicates(subset='Text Clean',inplace=True)
len(dt_prep)

20525

In [15]:
dt_prep.isna().sum()

Date          0
Text          0
Label         0
Text Clean    0
dtype: int64

### Meaningless

In [16]:
meaningless = pd.read_csv("Kamus Normalisasi + Meaninglessmeaningless.csv",header=None,names=['meaninglessStopword'])
meaningless = meaningless['meaninglessStopword'].tolist()
meaningless

['kamu',
 'aduh',
 'ah',
 'ahelah',
 'amp',
 'aw',
 'aww',
 'beb',
 'bebb',
 'bete',
 'btw',
 'cyiinn',
 'cyin',
 'dah',
 'deh',
 'doang',
 'duh',
 'duhhh',
 'eeh',
 'eh',
 'ehh',
 'ew',
 'ewh',
 'ftw',
 'haduh',
 'ha',
 'hah',
 'haha',
 'hahaa',
 'hahaha',
 'hahahaa',
 'hahh',
 'hahhaa',
 'hahhahah',
 'he',
 'hehe',
 'hehehe',
 'hi',
 'hihi',
 'hihihi',
 'hiks',
 'hikss',
 'hiksss',
 'hmm',
 'hmmm',
 'hmmmm',
 'huft',
 'hufttt',
 'huhu',
 'huhuhu',
 'huhuhuu',
 'huhuu',
 'ih',
 'iiih',
 'iih',
 'ihhh',
 'ihh',
 'ko',
 'kog',
 'kok',
 'laah',
 'lah',
 'lebay',
 'lel',
 'lha',
 'lho',
 'lmao',
 'loh',
 'lohh',
 'lol',
 'looh',
 'muuaahh',
 'mwah',
 'mwahh',
 'nbsp',
 'nya',
 'nyaww',
 'oo',
 'ooo',
 'ooohh',
 'ooh',
 'oh',
 'ohh',
 'okayy',
 'okay',
 'okee',
 'okey',
 'okei',
 'oki',
 'oke',
 'ok',
 'ohhh',
 'ohiya',
 'ohya',
 'oiya',
 'oiyaa',
 'omg',
 'otw',
 'quot',
 'sayy',
 'si',
 'supeeerrr',
 'super',
 'superr',
 'waduh',
 'waduh',
 'waduhh',
 'wah',
 'waw',
 'wew',
 'wih',
 'wkw

In [17]:
def removeMeaningless(data):
    meaningless = pd.read_csv("Kamus Normalisasi + Meaninglessmeaningless.csv",header=None,names=['stopword'])
    meaningless = meaningless['stopword'].tolist()
    for word in meaningless:
        data = re.sub(r"\b" + word + r"\b" , " ",data)
    return data

In [18]:
dt_prep['meaningless'] = dt_prep['Text Clean'].apply(removeMeaningless)
dt_prep

,Date,Text,Label,Text Clean,meaningless
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...,Bagaimana gk ada pemberitahuan kpn selesainy...
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...,HOREE awal saldo nol Setelah cek jt gue tarik ...
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya,Bankerot nih namanya
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...,Alhandulillah Rek Mandiri amaann kemarin
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri normal lagi urus...
...,...,...,...,...,...
22124,2019-04-02,@GdeDyaksa Terima kasih Bpk Dyaksa atas partis...,1,Terima kasih Bpk Dyaksa atas partisipasi yg di...,Terima kasih Bpk Dyaksa atas partisipasi dib...
22125,2019-04-02,@mohammadiball Terima kasih Bpk Ibal atas part...,1,Terima kasih Bpk Ibal atas partisipasi yg dibe...,Terima kasih Bpk Ibal atas partisipasi diber...
22126,2019-04-02,@JuraganHusin Terima kasih Bpk Husin atas part...,1,Terima kasih Bpk Husin atas partisipasi yg dib...,Terima kasih Bpk Husin atas partisipasi dibe...
22127,2019-04-02,@vill4nk Terima kasih Bpk/Ibu atas partisipasi...,1,nk Terima kasih Bpk Ibu atas partisipasi yg di...,nk Terima kasih Bpk Ibu atas partisipasi dib...


In [19]:
dt_prep.to_csv('Data Clean.csv', header = True, index = False)

In [20]:
dt_prep.reset_index(drop=True,inplace = True)
dt_prep

,Date,Text,Label,Text Clean,meaningless
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...,Bagaimana gk ada pemberitahuan kpn selesainy...
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...,HOREE awal saldo nol Setelah cek jt gue tarik ...
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya,Bankerot nih namanya
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...,Alhandulillah Rek Mandiri amaann kemarin
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri normal lagi urus...
...,...,...,...,...,...
20520,2019-04-02,@GdeDyaksa Terima kasih Bpk Dyaksa atas partis...,1,Terima kasih Bpk Dyaksa atas partisipasi yg di...,Terima kasih Bpk Dyaksa atas partisipasi dib...
20521,2019-04-02,@mohammadiball Terima kasih Bpk Ibal atas part...,1,Terima kasih Bpk Ibal atas partisipasi yg dibe...,Terima kasih Bpk Ibal atas partisipasi diber...
20522,2019-04-02,@JuraganHusin Terima kasih Bpk Husin atas part...,1,Terima kasih Bpk Husin atas partisipasi yg dib...,Terima kasih Bpk Husin atas partisipasi dibe...
20523,2019-04-02,@vill4nk Terima kasih Bpk/Ibu atas partisipasi...,1,nk Terima kasih Bpk Ibu atas partisipasi yg di...,nk Terima kasih Bpk Ibu atas partisipasi dib...


# Preprocessing

In [21]:
dt_prep = dt_prep.copy()
dt_prep

,Date,Text,Label,Text Clean,meaningless
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...,Bagaimana gk ada pemberitahuan kpn selesainy...
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...,HOREE awal saldo nol Setelah cek jt gue tarik ...
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya,Bankerot nih namanya
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...,Alhandulillah Rek Mandiri amaann kemarin
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri normal lagi urus...
...,...,...,...,...,...
20520,2019-04-02,@GdeDyaksa Terima kasih Bpk Dyaksa atas partis...,1,Terima kasih Bpk Dyaksa atas partisipasi yg di...,Terima kasih Bpk Dyaksa atas partisipasi dib...
20521,2019-04-02,@mohammadiball Terima kasih Bpk Ibal atas part...,1,Terima kasih Bpk Ibal atas partisipasi yg dibe...,Terima kasih Bpk Ibal atas partisipasi diber...
20522,2019-04-02,@JuraganHusin Terima kasih Bpk Husin atas part...,1,Terima kasih Bpk Husin atas partisipasi yg dib...,Terima kasih Bpk Husin atas partisipasi dibe...
20523,2019-04-02,@vill4nk Terima kasih Bpk/Ibu atas partisipasi...,1,nk Terima kasih Bpk Ibu atas partisipasi yg di...,nk Terima kasih Bpk Ibu atas partisipasi dib...


### Case Folding

In [22]:
dt_prep['Case Folding'] = dt_prep['Text Clean'].str.lower()
dt_prep.head()

,Date,Text,Label,Text Clean,meaningless,Case Folding
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...,Bagaimana gk ada pemberitahuan kpn selesainy...,bagaimana ini gk ada pemberitahuan kpn selesai...
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...,HOREE awal saldo nol Setelah cek jt gue tarik ...,horee awal saldo nol setelah cek jt gue tarik ...
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya,Bankerot nih namanya,bankerot nih namanya
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...,Alhandulillah Rek Mandiri amaann kemarin,alhandulillah rek mandiri saya amaann masih se...
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri normal lagi urus...,mohon infonya kapan mandiri bisa normal lagi u...


### Data Normalization

In [23]:
# READ KAMUS GAB
kamus = pd.read_csv("Kamus Normalisasi + Meaninglesskamus_gab.csv", header=None,names=["Before","After"], encoding ='latin1')
kamus.tail()

,Before,After
17181,soalny,soal
17182,tolonglah,tolong
17183,ktidaknyamanannya,ketidak nyamanan
17184,kgagalan,gagal
17185,ktahuan,ketahuan


In [24]:
nonstpd = kamus['Before'].values.tolist()
stpd = kamus['After'].values.tolist()

In [25]:
def normalisasitext(text, after, before):
    text = text.split(" ")
    for i in range(len(text)):
        if text[i] in before:
            index = before.index(text[i])
            text[i] = after[index]
    return ' '.join(map(str, text))

def normalizeData(data, after, before):
    data = data.map(lambda com : normalisasitext(com, after, before))
    return data

In [26]:
dt_prep['normalisasi'] = normalizeData(dt_prep['Case Folding'], stpd, nonstpd)
dt_prep

,Date,Text,Label,Text Clean,meaningless,Case Folding,normalisasi
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...,Bagaimana gk ada pemberitahuan kpn selesainy...,bagaimana ini gk ada pemberitahuan kpn selesai...,bagaimana ini tidak ada pemberitahuan kapan se...
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...,HOREE awal saldo nol Setelah cek jt gue tarik ...,horee awal saldo nol setelah cek jt gue tarik ...,hore awal saldo nol setelah cek juta saya tari...
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya,Bankerot nih namanya,bankerot nih namanya,bangkrut ini namanya
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...,Alhandulillah Rek Mandiri amaann kemarin,alhandulillah rek mandiri saya amaann masih se...,alhamdulillah rekening mandiri saya aman masih...
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri normal lagi urus...,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri bisa normal lagi u...
...,...,...,...,...,...,...,...
20520,2019-04-02,@GdeDyaksa Terima kasih Bpk Dyaksa atas partis...,1,Terima kasih Bpk Dyaksa atas partisipasi yg di...,Terima kasih Bpk Dyaksa atas partisipasi dib...,terima kasih bpk dyaksa atas partisipasi yg di...,terima kasih bapak dyaksa atas partisipasi yan...
20521,2019-04-02,@mohammadiball Terima kasih Bpk Ibal atas part...,1,Terima kasih Bpk Ibal atas partisipasi yg dibe...,Terima kasih Bpk Ibal atas partisipasi diber...,terima kasih bpk ibal atas partisipasi yg dibe...,terima kasih bapak ibal atas partisipasi yang ...
20522,2019-04-02,@JuraganHusin Terima kasih Bpk Husin atas part...,1,Terima kasih Bpk Husin atas partisipasi yg dib...,Terima kasih Bpk Husin atas partisipasi dibe...,terima kasih bpk husin atas partisipasi yg dib...,terima kasih bapak husin atas partisipasi yang...
20523,2019-04-02,@vill4nk Terima kasih Bpk/Ibu atas partisipasi...,1,nk Terima kasih Bpk Ibu atas partisipasi yg di...,nk Terima kasih Bpk Ibu atas partisipasi dib...,nk terima kasih bpk ibu atas partisipasi yg di...,natural killer terima kasih bapak ibu atas par...


## Stop Word

In [27]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
 
Stopword = StopWordRemoverFactory() #kata yang ada pasa sastrawi Indonesia
list_word = Stopword.get_stop_words()
word = Stopword.create_stop_word_remover()

print(list_word)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [28]:
# FUNGSI REMOVE STOP WORDS
def remove_stopword(data,stop_words_):
    for stop_word in stop_words_:
        regex_stopword = r"\b(?:{})\b".format('|'.join(stop_word))
        data = data.str.replace(regex_stopword, '')
        # data = re.sub("\s+", " ",data)
    return data

In [29]:
dt_prep['Stopword'] = remove_stopword(dt_prep['normalisasi'], list_word).apply(cleaning) # no stemming, stopword
dt_prep.head(10)

,Date,Text,Label,Text Clean,meaningless,Case Folding,normalisasi,Stopword
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...,Bagaimana gk ada pemberitahuan kpn selesainy...,bagaimana ini gk ada pemberitahuan kpn selesai...,bagaimana ini tidak ada pemberitahuan kapan se...,bagaimana ini tidak ada pemberitahuan kapan se...
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...,HOREE awal saldo nol Setelah cek jt gue tarik ...,horee awal saldo nol setelah cek jt gue tarik ...,hore awal saldo nol setelah cek juta saya tari...,hore awal saldo nol setelah cek juta saya tari...
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya,Bankerot nih namanya,bankerot nih namanya,bangkrut ini namanya,bangkrut ini namanya
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...,Alhandulillah Rek Mandiri amaann kemarin,alhandulillah rek mandiri saya amaann masih se...,alhamdulillah rekening mandiri saya aman masih...,alhamdulillah rekening mandiri saya aman masih...
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri normal lagi urus...,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri bisa normal lagi u...
5,2019-07-20,saldo saya bertambah Rp. 21.500.000 gimana nih...,-1,saldo saya bertambah Rp gimana nih saya gak ma...,saldo bertambah Rp gimana nih gak mau moho...,saldo saya bertambah rp gimana nih saya gak ma...,saldo saya bertambah rupiah bagaimana ini saya...,saldo saya bertambah rupiah bagaimana ini saya...
6,2019-07-20,Cek saldo mandiri kok gak bisa-bisa mulu. Baru...,0,Cek saldo mandiri kok gak bisa mulu Baru inget...,Cek saldo mandiri gak mulu Baru inget kala...,cek saldo mandiri kok gak bisa mulu baru inget...,cek saldo mandiri kok tidak bisa melulu baru i...,cek saldo mandiri kok tidak bisa melulu baru i...
7,2019-07-20,ðŸ˜Œ15 juta hilang uang nikah q parah bgt tolo...,-1,juta hilang uang nikah q parah bgt tolong dong...,juta hilang uang nikah q parah bgt tolong se...,juta hilang uang nikah q parah bgt tolong dong...,juta hilang uang nikah aku parah banget tolong...,juta hilang uang nikah aku parah banget tolong...
8,2019-07-20,Ane gan. Saldo ilang abis smua. Hadeuhhhh. Moh...,-1,Ane gan Saldo ilang abis smua Hadeuhh Mohon pe...,Ane gan Saldo ilang smua Hadeuhh Mohon pence...,ane gan saldo ilang abis smua hadeuhh mohon pe...,saya juragan saldo hilang habis semua aduh moh...,saya juragan saldo hilang habis semua aduh moh...
9,2019-07-20,sabtu adalah saatnya bagi kami menggaji karyaw...,-1,sabtu adalah saatnya bagi kami menggaji karyaw...,sabtu saatnya menggaji karyawan dana a...,sabtu adalah saatnya bagi kami menggaji karyaw...,sabtu adalah saatnya bagi kami menggaji karyaw...,sabtu adalah saatnya bagi kami menggaji karyaw...


## Stemming

In [30]:
stemmer = StemmerFactory().create_stemmer() # sastrawi stemmer

In [31]:
dt_prep['Stemming'] = stemData(dt_prep['Stopword'],stemmer)

In [32]:
dt_prep

,Date,Text,Label,Text Clean,meaningless,Case Folding,normalisasi,Stopword,Stemming
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...,Bagaimana gk ada pemberitahuan kpn selesainy...,bagaimana ini gk ada pemberitahuan kpn selesai...,bagaimana ini tidak ada pemberitahuan kapan se...,bagaimana ini tidak ada pemberitahuan kapan se...,bagaimana ini tidak ada pemberitahuan kapan se...
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...,HOREE awal saldo nol Setelah cek jt gue tarik ...,horee awal saldo nol setelah cek jt gue tarik ...,hore awal saldo nol setelah cek juta saya tari...,hore awal saldo nol setelah cek juta saya tari...,hore awal saldo nol telah cek juta saya tarik ...
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya,Bankerot nih namanya,bankerot nih namanya,bangkrut ini namanya,bangkrut ini namanya,bangkrut ini nama
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...,Alhandulillah Rek Mandiri amaann kemarin,alhandulillah rek mandiri saya amaann masih se...,alhamdulillah rekening mandiri saya aman masih...,alhamdulillah rekening mandiri saya aman masih...,alhamdulillah rekening mandiri saya aman masih...
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri normal lagi urus...,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri bisa normal lagi u...,mohon info kapan mandiri bisa normal lagi urus...
...,...,...,...,...,...,...,...,...,...
20520,2019-04-02,@GdeDyaksa Terima kasih Bpk Dyaksa atas partis...,1,Terima kasih Bpk Dyaksa atas partisipasi yg di...,Terima kasih Bpk Dyaksa atas partisipasi dib...,terima kasih bpk dyaksa atas partisipasi yg di...,terima kasih bapak dyaksa atas partisipasi yan...,terima kasih bapak dyaksa atas partisipasi yan...,terima kasih bapak dyaksa atas partisipasi yan...
20521,2019-04-02,@mohammadiball Terima kasih Bpk Ibal atas part...,1,Terima kasih Bpk Ibal atas partisipasi yg dibe...,Terima kasih Bpk Ibal atas partisipasi diber...,terima kasih bpk ibal atas partisipasi yg dibe...,terima kasih bapak ibal atas partisipasi yang ...,terima kasih bapak ibal atas partisipasi yang ...,terima kasih bapak ibal atas partisipasi yang ...
20522,2019-04-02,@JuraganHusin Terima kasih Bpk Husin atas part...,1,Terima kasih Bpk Husin atas partisipasi yg dib...,Terima kasih Bpk Husin atas partisipasi dibe...,terima kasih bpk husin atas partisipasi yg dib...,terima kasih bapak husin atas partisipasi yang...,terima kasih bapak husin atas partisipasi yang...,terima kasih bapak husin atas partisipasi yang...
20523,2019-04-02,@vill4nk Terima kasih Bpk/Ibu atas partisipasi...,1,nk Terima kasih Bpk Ibu atas partisipasi yg di...,nk Terima kasih Bpk Ibu atas partisipasi dib...,nk terima kasih bpk ibu atas partisipasi yg di...,natural killer terima kasih bapak ibu atas par...,natural killer terima kasih bapak ibu atas par...,natural killer terima kasih bapak ibu atas par...


In [33]:
dup2 = dt_prep.duplicated()
dup2.sum()

0

## Tokenization

In [34]:
dt_prep['tokenizing'] = dt_prep['Stemming'].apply(tokenizing)

In [35]:
dt_prep.reset_index(drop=True,inplace = True)
dt_prep

,Date,Text,Label,Text Clean,meaningless,Case Folding,normalisasi,Stopword,Stemming,tokenizing
0,2019-07-20,Bagaimana ini gk ada pemberitahuan kpn selesai...,-1,Bagaimana ini gk ada pemberitahuan kpn selesai...,Bagaimana gk ada pemberitahuan kpn selesainy...,bagaimana ini gk ada pemberitahuan kpn selesai...,bagaimana ini tidak ada pemberitahuan kapan se...,bagaimana ini tidak ada pemberitahuan kapan se...,bagaimana ini tidak ada pemberitahuan kapan se...,"[bagaimana, ini, tidak, ada, pemberitahuan, ka..."
1,2019-07-20,HOREEE. awal saldo nol. Setelah cek 175jt. gue...,0,HOREE awal saldo nol Setelah cek jt gue tarik ...,HOREE awal saldo nol Setelah cek jt gue tarik ...,horee awal saldo nol setelah cek jt gue tarik ...,hore awal saldo nol setelah cek juta saya tari...,hore awal saldo nol setelah cek juta saya tari...,hore awal saldo nol telah cek juta saya tarik ...,"[hore, awal, saldo, nol, telah, cek, juta, say..."
2,2019-07-20,Bankerot nih namanya,-1,Bankerot nih namanya,Bankerot nih namanya,bankerot nih namanya,bangkrut ini namanya,bangkrut ini namanya,bangkrut ini nama,"[bangkrut, ini, nama]"
3,2019-07-20,Alhandulillah . . . .Rek Mandiri saya amaannn ...,1,Alhandulillah Rek Mandiri saya amaann masih se...,Alhandulillah Rek Mandiri amaann kemarin,alhandulillah rek mandiri saya amaann masih se...,alhamdulillah rekening mandiri saya aman masih...,alhamdulillah rekening mandiri saya aman masih...,alhamdulillah rekening mandiri saya aman masih...,"[alhamdulillah, rekening, mandiri, saya, aman,..."
4,2019-07-20,mohon infonya kapan mandiri bisa normal lagi u...,0,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri normal lagi urus...,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri bisa normal lagi u...,mohon infonya kapan mandiri bisa normal lagi u...,mohon info kapan mandiri bisa normal lagi urus...,"[mohon, info, kapan, mandiri, bisa, normal, la..."
...,...,...,...,...,...,...,...,...,...,...
20520,2019-04-02,@GdeDyaksa Terima kasih Bpk Dyaksa atas partis...,1,Terima kasih Bpk Dyaksa atas partisipasi yg di...,Terima kasih Bpk Dyaksa atas partisipasi dib...,terima kasih bpk dyaksa atas partisipasi yg di...,terima kasih bapak dyaksa atas partisipasi yan...,terima kasih bapak dyaksa atas partisipasi yan...,terima kasih bapak dyaksa atas partisipasi yan...,"[terima, kasih, bapak, dyaksa, atas, partisipa..."
20521,2019-04-02,@mohammadiball Terima kasih Bpk Ibal atas part...,1,Terima kasih Bpk Ibal atas partisipasi yg dibe...,Terima kasih Bpk Ibal atas partisipasi diber...,terima kasih bpk ibal atas partisipasi yg dibe...,terima kasih bapak ibal atas partisipasi yang ...,terima kasih bapak ibal atas partisipasi yang ...,terima kasih bapak ibal atas partisipasi yang ...,"[terima, kasih, bapak, ibal, atas, partisipasi..."
20522,2019-04-02,@JuraganHusin Terima kasih Bpk Husin atas part...,1,Terima kasih Bpk Husin atas partisipasi yg dib...,Terima kasih Bpk Husin atas partisipasi dibe...,terima kasih bpk husin atas partisipasi yg dib...,terima kasih bapak husin atas partisipasi yang...,terima kasih bapak husin atas partisipasi yang...,terima kasih bapak husin atas partisipasi yang...,"[terima, kasih, bapak, husin, atas, partisipas..."
20523,2019-04-02,@vill4nk Terima kasih Bpk/Ibu atas partisipasi...,1,nk Terima kasih Bpk Ibu atas partisipasi yg di...,nk Terima kasih Bpk Ibu atas partisipasi dib...,nk terima kasih bpk ibu atas partisipasi yg di...,natural killer terima kasih bapak ibu atas par...,natural killer terima kasih bapak ibu atas par...,natural killer terima kasih bapak ibu atas par...,"[natural, killer, terima, kasih, bapak, ibu, a..."


In [36]:
dt_prep.isna().sum()

Date            0
Text            0
Label           0
Text Clean      0
meaningless     0
Case Folding    0
normalisasi     0
Stopword        0
Stemming        0
tokenizing      0
dtype: int64

In [38]:
dt_prep.to_csv('new_Data - Bismillah - Stemming.csv', header = True, index = False)